# Import Packages And Functions

In [1]:
# 🛠 Monkey patch torch.load to fix PyTorch 2.6+ issue with weights_only=True
import torch
old_load = torch.load
torch.load = lambda *args, **kwargs: old_load(*args, **{**kwargs, 'weights_only': False})

# 📥 Load the dataset from OGB
from ogb.nodeproppred import NodePropPredDataset

# Load ogbn-proteins graph and labels
dataset = NodePropPredDataset(name='ogbn-proteins')
graph, labels = dataset[0]

# 🎯 Extract edges and one feature as weights
edge_index = graph['edge_index']  # shape: [2, num_edges]
edge_feat = graph['edge_feat']    # shape: [num_edges, 8]

# Use the first edge feature (or mean across all features)
weights = edge_feat[:, 0]  # ✅ Already a NumPy array

# 💾 Save to file in your expected format: "source target weight"
with open('../data/edges_ogbn-proteins.txt', 'w') as f:
    for i in range(edge_index.shape[1]):
        src = int(edge_index[0, i])
        dst = int(edge_index[1, i])
        wt = float(weights[i])
        f.write(f"{src} {dst} {wt:.6f}\n")

print("✅ Dataset converted and saved as 'edges_ogbn-proteins.txt'")


✅ Dataset converted and saved as 'edges_ogbn-proteins.txt'


In [2]:
# Packages
import sys; sys.path.append('..')
from Code.data import data_load
import Code.influence as influence
import Code.block_edges as be
import Code.communities as communities
import networkx as nx

# Load The Dataset

In [1]:
# Required libraries
from ogb.nodeproppred import NodePropPredDataset
import torch

# Monkey patch to fix PyTorch 2.6+ weights_only issue
old_load = torch.load
torch.load = lambda *args, **kwargs: old_load(*args, **{**kwargs, 'weights_only': False})

# Load OGBN-Proteins dataset
dataset = NodePropPredDataset(name='ogbn-proteins')
graph, labels = dataset[0]

edge_index = graph['edge_index']
edge_feat = graph['edge_feat']
weights = edge_feat[:, 0]  # First feature as weight

# Limit to first 50,000 edges to avoid MemoryError
with open('../data/edges_ogbn-proteins.txt', 'w') as f:
    for i in range(4000):
        src = int(edge_index[0, i])
        dst = int(edge_index[1, i])
        wt = float(weights[i])
        f.write(f"{src} {dst} {wt:.6f}\n")

print("✅ Saved reduced dataset with 4000 edges.")


✅ Saved reduced dataset with 4000 edges.


# Average Number Of Nodes Infected Initially

In [2]:
import sys
sys.path.append('..')  # Only needed if you're outside the "Code" folder

from Code import influence  # ✅ This will make influence.* available


In [3]:
import sys
sys.path.append('..')  # go one level up where "Code" folder is

import networkx as nx
from Code.data import data_load  # your existing loader

# Load the reduced dataset from file
path = '../data/edges_ogbn-proteins.txt'
nodes, edges, weights = data_load(path)

# Now build the graph safely
graph = nx.Graph()
min_wt = min(weights.values())
eps = min_wt * 0.001
wt_factor = 1

for e in edges:
    graph.add_edge(e[0], e[1],
                   weight=weights[e] / wt_factor,
                   distance=1 - weights[e] + eps)

print("✅ Graph created:", len(graph.nodes), "nodes,", len(graph.edges), "edges")


Number of Nodes: 2037
Number of Edges: 4000
✅ Graph created: 2037 nodes, 2000 edges


In [4]:
type(graph)


networkx.classes.graph.Graph

In [5]:
from Code import influence  # Import your influence module

# Set parameters
init_rate = 0.01
iters = 50

# Run simulation
a = influence.influence_count_multiple(graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# Output result
print("✅ ICM Simulation Completed")
print("Nodes Infected =", a)
print("Infection Percentage = {:.2f}%".format(b))


✅ ICM Simulation Completed
Nodes Infected = 464.38
Infection Percentage = 22.80%


In [44]:

from Code import influence  # Import your influence module
import random

# Set parameters
init_rate = 0.01  # 1% initially infected
iters = 50

# Convert graph.nodes to a list
nodes_list = list(graph.nodes)

# Identify initial infected nodes
num_initial_infected = int(len(nodes_list) * init_rate)
initial_infected_nodes = random.sample(nodes_list, num_initial_infected)

# Run simulation (this will simulate infection propagation)
a = influence.influence_count_multiple(graph, init_rate, iters)

# Calculate infection percentage
b = 100 * (a / len(graph.nodes))

# Output results
print("✅ ICM Simulation Completed")
print("Initial Infected Nodes:", initial_infected_nodes)  # Show initial infected nodes
print("Nodes Infected =", a)
print("Infection Percentage = {:.2f}%".format(b))



✅ ICM Simulation Completed
Initial Infected Nodes: [29264, 75601, 34657, 52635, 104025, 69754, 60076, 54037, 71584, 65508, 102751, 86573, 129206, 117364, 86418, 42290, 69051, 33869, 66260, 90086]
Nodes Infected = 536.54
Infection Percentage = 26.34%


# RNDM Method
Randomly removes edges

In [6]:
from Code import block_edges as be

# Random blocking
output = be.RNDM(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

print("🧱 After Blocking Edges Randomly:")
print("Nodes Infected =", a)
print("Infection Percentage = {:.2f}%".format(b))


🧱 After Blocking Edges Randomly:
Nodes Infected = 523.64
Infection Percentage = 25.71%


# HWT Method
Removes edges with the highest weight

In [7]:
from Code import block_edges as be

# 🔧 Apply HWGT method (block edges with highest weights)
output = be.HWGT(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("🔗 After Blocking Edges With Highest Weight (HWGT):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


🔗 After Blocking Edges With Highest Weight (HWGT):
Nodes Infected = 564.44
Infection Percentage = 27.71%


# DEG Method
Blocks edges where the sum of degrees of adjacent nodes is highest.

In [8]:
from Code import block_edges as be

# 🔧 Apply HDEG method (block edges with highest sum of adjacent node degrees)
output = be.HDEG(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("📐 After Blocking Edges With Highest Degree Sum (HDEG):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


📐 After Blocking Edges With Highest Degree Sum (HDEG):
Nodes Infected = 520.92
Infection Percentage = 25.57%


# WDEG Method
Blocks edges with the highest weighted sum of degrees.

In [9]:
from Code import block_edges as be

# 🔧 Apply WDEG method (block edges with highest sum of weighted degrees of adjacent nodes)
output = be.WDEG(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("⚖️ After Blocking Edges With Highest Weighted Degree Sum (WDEG):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


⚖️ After Blocking Edges With Highest Weighted Degree Sum (WDEG):
Nodes Infected = 510.12
Infection Percentage = 25.04%


# CLO Method
Blocks edges between central nodes.

In [10]:
from Code import block_edges as be

# 🔧 Apply CLO method (block edges with highest sum of closeness of adjacent nodes)
output = be.CLO(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("📍 After Blocking Edges With Highest Closeness Centrality Sum (CLO):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


📍 After Blocking Edges With Highest Closeness Centrality Sum (CLO):
Nodes Infected = 543.20
Infection Percentage = 26.67%


# WCLO Method
Blocks edges where the weighted closeness of adjacent nodes is high.

In [11]:
from Code import block_edges as be

# 🔧 Apply WCLO method (block edges with highest sum of weighted closeness of adjacent nodes)
output = be.WCLO(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("📶 After Blocking Edges With Highest Weighted Closeness Centrality Sum (WCLO):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


📶 After Blocking Edges With Highest Weighted Closeness Centrality Sum (WCLO):
Nodes Infected = 557.04
Infection Percentage = 27.35%


# BET Method
Blocks edges with highest betweenness centrality.

In [12]:
from Code import block_edges as be

# 🔧 Apply BET method (block edges with highest betweenness centrality)
output = be.BTWN(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("🔀 After Blocking Edges With Highest Betweenness Centrality (BET):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


🔀 After Blocking Edges With Highest Betweenness Centrality (BET):
Nodes Infected = 521.74
Infection Percentage = 25.61%


# WBET Method
Blocks high-weight edges with high betweenness.

In [13]:
from Code import block_edges as be

# 🔧 Apply WBET method (block edges with highest weighted betweenness centrality)
output = be.WBET(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("💡 After Blocking Edges With Highest Weighted Betweenness Centrality (WBET):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


💡 After Blocking Edges With Highest Weighted Betweenness Centrality (WBET):
Nodes Infected = 470.56
Infection Percentage = 23.10%


# PGRK Method
Blocks edges between nodes with high PageRank scores.

In [14]:
from Code import block_edges as be

# 🔧 Apply PGRK method (block edges between nodes with highest combined PageRank)
output = be.PGRK(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k=0.01)

# 🧪 Run the ICM after blocking
a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

# 📊 Display results
print("📎 After Blocking Edges With Highest Sum of PageRank of Adjacent Nodes (PGRK):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


📎 After Blocking Edges With Highest Sum of PageRank of Adjacent Nodes (PGRK):
Nodes Infected = 608.88
Infection Percentage = 29.89%


# IEED Method
Uses an iterative entropy-based approach.

In [15]:
from Code import block_edges as be
import networkx as nx
k = 0.01
# 🔧 Apply IEED method (remove edges using entropy and efficiency-based ranking)
edge_list = be.IEED_remove_edges(nodes.copy(), edges.copy(), weights, k)

# 🌐 Rebuild the graph after removing selected edges
final_graph = nx.Graph()
min_wt = min(weights.values())
eps = min_wt * 0.001
wt_factor = 1

for e in edge_list:
    final_graph.add_edge(e[0], e[1],
                         weight=weights[e] / wt_factor,
                         distance=1 - weights[e] + eps)

# 🧪 Run ICM after blocking
a = influence.influence_count_multiple(final_graph, init_rate, iters)
b = 100 * (a / len(nodes))

# 📊 Display results
print("🧠 After Blocking Edges Using Iterative Entropy-Based Edge Deletion (IEED):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


Edges removed:  40
🧠 After Blocking Edges Using Iterative Entropy-Based Edge Deletion (IEED):
Nodes Infected = 526.76
Infection Percentage = 25.86%


In [16]:
# K-Core Edge Blocking
from Code import block_edges as be
output = be.KCORE(graph)
blocked_graph = be.rem_edges(graph.copy(), output, k)

a = influence.influence_count_multiple(blocked_graph, init_rate, iters)
b = 100 * (a / len(nodes))
print("🌐 After Blocking Edges Connecting High K-Core Nodes:")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


🌐 After Blocking Edges Connecting High K-Core Nodes:
Nodes Infected = 442.84
Infection Percentage = 21.74%


In [20]:
#Hybrid Blocking (IEED + Community):
from Code import communities
# ✅ Parameters to tune
init_rate = 0.0005     # Smaller fraction of seed nodes
k = 0.03               # Block 3% of edges instead of 1%
iters = 10             # Number of ICM simulation runs

# ✅ Get IEED and Community-based edges
ieed_edges = be.IEED_remove_edges(nodes.copy(), edges.copy(), weights, k)
comm = communities.detect_communities(graph, 0.15)
conn_edges, _, _ = communities.connecting_edges(comm, graph)

# ✅ Combine both edge lists (union OR intersection)
# Option A: Use union of edges (more aggressive)
combined = list(set(ieed_edges + conn_edges))

# Option B: Use only overlapping critical edges (more selective)
# combined = list(set(ieed_edges).intersection(set(conn_edges)))

# ✅ Trim to top-k fraction
combined = combined[:int(k * len(edges))]

# ✅ Remove selected edges from graph
hybrid_graph = graph.copy()
hybrid_graph.remove_edges_from(combined)

# ✅ Run ICM after edge removal
a = influence.influence_count_multiple(hybrid_graph, init_rate, iters)
b = 100 * (a / len(nodes))

# ✅ Output results
print("🧬 Final Hybrid Blocking (IEED + Community):")
print(f"Edges Removed = {len(combined)}")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


Edges removed:  120
🧬 Final Hybrid Blocking (IEED + Community):
Edges Removed = 120
Nodes Infected = 28.10
Infection Percentage = 1.38%


In [22]:
# EDGE WEIGHT REDUCTION (Soft Blocking)
def apply_weight_soft_blocking(graph, edge_list, factor=0.2):
    reduced_graph = graph.copy()
    for u, v in edge_list:
        if reduced_graph.has_edge(u, v):
            reduced_graph[u][v]['weight'] *= factor
            reduced_graph[u][v]['distance'] = 1 - reduced_graph[u][v]['weight']
    return reduced_graph

output = be.HDEG(graph)  # or any other method to get edge list
soft_blocked_graph = apply_weight_soft_blocking(graph, output, factor=0.2)

a = influence.influence_count_multiple(soft_blocked_graph, init_rate, iters)
b = 100 * (a / len(graph.nodes))

print("💧 Soft Blocking via Weight Reduction:")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")

💧 Soft Blocking via Weight Reduction:
Nodes Infected = 1.50
Infection Percentage = 0.07%


In [23]:
# ✅ VERIFIER ASSIGNMENT STRATEGY

# Step 1: Find Connecting Edges Between Communities
def detect_connecting_edges(graph, resolution=0.15):
    comm = communities.detect_communities(graph, resolution)
    conn_edges, _, _ = communities.connecting_edges(comm, graph)
    return conn_edges

# Step 2: Assign Verifiers to Key Connecting Nodes
def assign_verifiers_on_connectors(graph, conn_edges, k):
    graph_conn = nx.Graph(conn_edges)
    deg = nx.degree_centrality(graph_conn)

    n = len(graph_conn.nodes()) - 1 if len(graph_conn.nodes()) > 1 else 1
    for i in deg:
        deg[i] = round(deg[i] * n)

    verifier_nodes = []
    remaining = 0
    for _ in range(int(k * len(graph.nodes()))):
        if not deg:
            break
        key_max = max(zip(deg.values(), deg.keys()))[1]
        if deg[key_max] == 0:
            remaining += 1
            break
        verifier_nodes.append(key_max)
        for neighbor in graph_conn.neighbors(key_max):
            deg[neighbor] -= 1
        graph_conn.remove_node(key_max)
        del deg[key_max]

    if remaining > 0:
        remaining_nodes = list(set(graph.nodes()) - set(verifier_nodes))
        verifier_nodes.extend(remaining_nodes[:remaining])

    for node in graph.nodes():
        graph.nodes[node]["verifier"] = 1 if node in verifier_nodes else 0

    return graph, verifier_nodes

# Step 3: Run Strategy + ICM Evaluation
def run_verifier_strategy(graph, init_rate=0.001, iters=10, k=0.01, resolution=0.15):
    conn_edges = detect_connecting_edges(graph, resolution)
    graph_with_verifiers, verifiers = assign_verifiers_on_connectors(graph.copy(), conn_edges, k)

    print(f"🔐 Verifiers Assigned to {len(verifiers)} Nodes")

    # Run ICM simulation
    a = influence.influence_count_multiple(graph_with_verifiers, init_rate, iters)
    b = 100 * (a / len(graph_with_verifiers.nodes))
    print("🧪 ICM Evaluation After Verifier Assignment:")
    print(f"Nodes Infected = {a:.2f}")
    print(f"Infection Percentage = {b:.2f}%")

    return graph_with_verifiers, a, b


In [26]:
def run_verifier_strategy(graph, init_rate=0.001, iters=10, k=0.01, resolution=0.15):
    conn_edges = detect_connecting_edges(graph, resolution)
    graph_with_verifiers, verifiers = assign_verifiers_on_connectors(graph.copy(), conn_edges, k)

    print(f"🔐 Verifiers Assigned to {len(verifiers)} Nodes")

    # Run ICM simulation
    a = influence.influence_count_multiple(graph_with_verifiers, init_rate, iters)
    b = 100 * (a / len(graph_with_verifiers.nodes))
    print("🧪 ICM Evaluation After Verifier Assignment:")
    print(f"Nodes Infected = {a:.2f}")
    print(f"Infection Percentage = {b:.2f}%")

    return graph_with_verifiers

# 🔁 Call it right after defining
run_verifier_strategy(graph, init_rate=0.001, iters=10, k=0.01)


🔐 Verifiers Assigned to 0 Nodes
🧪 ICM Evaluation After Verifier Assignment:
Nodes Infected = 87.00
Infection Percentage = 4.27%


In [28]:
# ✅ Required Imports
import networkx as nx
import random
from Code import influence, communities

# ✅ Function to Rewire Edges
def edge_redirection_with_icm(graph, conn_edges, init_rate=0.001, iters=10, num_to_rewire=50):
    graph = graph.copy()
    rewired = 0

    # Find potential redirect targets: verifiers or low-degree nodes
    verifiers = [n for n in graph.nodes if graph.nodes[n].get("verifier") == 1]
    low_deg_nodes = sorted(graph.degree, key=lambda x: x[1])[:len(graph) // 10]
    low_influence_targets = [n for n, _ in low_deg_nodes]

    for u, v in conn_edges:
        if rewired >= num_to_rewire or not graph.has_edge(u, v):
            continue
        graph.remove_edge(u, v)
        target = random.choice(verifiers or low_influence_targets)
        graph.add_edge(u, target, weight=0.1, distance=1 - 0.1 + 0.001)
        rewired += 1

    print(f"🔄 Rewired {rewired} edges to verifiers/low-degree nodes")

    # ✅ Run ICM simulation
    a = influence.influence_count_multiple(graph, init_rate, iters)
    b = 100 * (a / len(graph.nodes))

    print("📉 ICM Evaluation After Rewiring:")
    print(f"Nodes Infected = {a:.2f}")
    print(f"Infection Percentage = {b:.2f}%")
    
    return graph

# ✅ Step 1: Detect community connecting edges
resolution = 0.15
comm = communities.detect_communities(graph, resolution)
conn_edges, _, _ = communities.connecting_edges(comm, graph)

# ✅ Step 2: Run edge redirection strategy with ICM
graph_after_rewire = edge_redirection_with_icm(
    graph,
    conn_edges=conn_edges,
    init_rate=0.0005,  # Set as needed
    iters=10,
    num_to_rewire=50
)


🔄 Rewired 0 edges to verifiers/low-degree nodes
📉 ICM Evaluation After Rewiring:
Nodes Infected = 9.20
Infection Percentage = 0.45%


# Community Method

## Community Detection And Edge Removal

In [40]:
from Code import communities
import networkx as nx

# 📊 Set resolution and increment factor (used for Louvain community detection)
resolution = 0.15
increasing_factor = 1.05

happened = 0
best_p_conn = 0
best_edges_to_remove = []

# 🔍 Iteratively find connecting edges between communities
while happened <= 5:
    for _ in range(5):
        comm = communities.detect_communities(graph, resolution)
        conn_edges, comm_edges, conn_nodes = communities.connecting_edges(comm, graph)
        p_conn = len(conn_edges) / len(edges)
        print(f"🔹 Fraction of connecting edges at resolution {resolution:.4f}: {p_conn:.4f}")
        
        if (p_conn <= k) and (p_conn > best_p_conn):
            best_edges_to_remove = conn_edges
            best_p_conn = p_conn
    
    resolution *= increasing_factor
    if p_conn > k:
        happened += 1
    print("➡️ Updated resolution:", round(resolution, 4))

# 🧱 Remove best connecting edges
final_graph = communities.graph_after_rem_edges(best_edges_to_remove, graph.copy())

# 🧪 Run ICM simulation
a = influence.influence_count_multiple(final_graph, init_rate, iters)
b = 100 * (a / len(nodes))

# 📊 Display result
print("🌐 After Blocking Edges Connecting Different Communities (Community Method):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


🔹 Fraction of connecting edges at resolution 0.1500: 0.0000
🔹 Fraction of connecting edges at resolution 0.1500: 0.0000
🔹 Fraction of connecting edges at resolution 0.1500: 0.0000
🔹 Fraction of connecting edges at resolution 0.1500: 0.0000
🔹 Fraction of connecting edges at resolution 0.1500: 0.0000
➡️ Updated resolution: 0.1575
🔹 Fraction of connecting edges at resolution 0.1575: 0.0000
🔹 Fraction of connecting edges at resolution 0.1575: 0.0000
🔹 Fraction of connecting edges at resolution 0.1575: 0.0000
🔹 Fraction of connecting edges at resolution 0.1575: 0.0000
🔹 Fraction of connecting edges at resolution 0.1575: 0.0000
➡️ Updated resolution: 0.1654
🔹 Fraction of connecting edges at resolution 0.1654: 0.0000
🔹 Fraction of connecting edges at resolution 0.1654: 0.0000
🔹 Fraction of connecting edges at resolution 0.1654: 0.0000
🔹 Fraction of connecting edges at resolution 0.1654: 0.0000
🔹 Fraction of connecting edges at resolution 0.1654: 0.0000
➡️ Updated resolution: 0.1736
🔹 Fraction

## ICM After Edge Removal

In [41]:
# 🧪 Run the Independent Cascade Model (ICM) after edge removal
a = influence.influence_count_multiple(final_graph, init_rate, iters)
b = 100 * (a / len(nodes))

# 📊 Display results
print("✅ ICM After Blocking Edges (Post-Removal Evaluation):")
print(f"Nodes Infected = {a:.2f}")
print(f"Infection Percentage = {b:.2f}%")


✅ ICM After Blocking Edges (Post-Removal Evaluation):
Nodes Infected = 25.10
Infection Percentage = 1.23%
